In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import multivariate_normal
pd.set_option('display.max_columns', None)  


In [2]:
mypath = 'Datasets/Gaze10minData_all/'
mydtaframes = {}

for index,files in enumerate(os.listdir(mypath)):
    mydtaframes["df_{}".format(index)] = pd.read_excel('{}/{}'.format(mypath, files),
                                                         sheet_name= 'Sheet1')

In [3]:
mydtaframes['df_1'].head()

,ParticipantName,RecordingDuration,RecordingTimestamp,LocalTimeStamp,EyeTrackerTimestamp,MouseEventIndex,MouseEventX (MCSpx),MouseEventY (MCSpx),KeyPressEventIndex,StudioEventIndex,StudioEventIndex_B,StudioEvent,StudioEvent_B,FixationIndex,SaccadeIndex,GazeEventType,GazeEventType_B,GazeEventDuration,FixationPointX (MCSpx),FixationPointY (MCSpx),AbsoluteSaccadicDirection,RelativeSaccadicDirection,GazePointIndex,GazePointX (MCSpx),GazePointY (MCSpx),PupilLeft,PupilRight,ValidityLeft,ValidityRight,StudioEventTypeDiff,GazeEventTypeDiff
0,Proband_31_D2,2895701,440,13:25:24.097,1.458043e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,NaN,2.0,Saccade,Saccade,133,NaN,NaN,NaN,NaN,21.0,66.0,220.0,"2,74","2,59",0.0,0.0,Agree_0_unstated,Agree_Saccade
1,Proband_31_D2,2895701,456,13:25:24.114,1.458043e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,NaN,2.0,Saccade,Saccade,133,NaN,NaN,NaN,NaN,22.0,20.0,166.0,"2,59","2,54",0.0,0.0,Agree_0_unstated,Agree_Saccade
2,Proband_31_D2,2895701,473,13:25:24.131,1.458043e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,2.0,NaN,Fixation,Fixation,200,45.0,168.0,"339,44",NaN,23.0,20.0,168.0,NaN,"2,62",4.0,0.0,Agree_0_unstated,Agree_Fixation
3,Proband_31_D2,2895701,490,13:25:24.147,1.458043e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,2.0,NaN,Fixation,Fixation,200,45.0,168.0,"339,44",NaN,24.0,36.0,171.0,"2,58","2,50",0.0,0.0,Agree_0_unstated,Agree_Fixation
4,Proband_31_D2,2895701,506,13:25:24.164,1.458043e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,2.0,NaN,Fixation,Fixation,200,45.0,168.0,"339,44",NaN,25.0,66.0,224.0,"2,74","2,60",0.0,0.0,Agree_0_unstated,Agree_Fixation


In [4]:
studiodf = {}

for index,frames in enumerate(mydtaframes):
    studiodf["{}".format(index)] = mydtaframes[frames]

In [5]:
len(studiodf)

20

In [6]:
for frames in studiodf:
    studiodf[frames].fillna(method='ffill', inplace= True)

In [7]:
allData = pd.concat([studiodf['0'],studiodf['1'],studiodf['2'],studiodf['3'],studiodf['4'],studiodf['5'],
                    studiodf['6'],studiodf['7'],studiodf['8'],studiodf['9'],studiodf['10'],studiodf['11'],
                    studiodf['12'],studiodf['13'],studiodf['14'],studiodf['15'],studiodf['16'],studiodf['17'],
                    studiodf['18'],studiodf['19'],], sort = False)
allData['GazePointX (MCSpx)'].fillna(method='bfill', inplace= True)
allData['GazePointY (MCSpx)'].fillna(method='bfill', inplace= True)

In [8]:
allData['GazePointX (MCSpx)'] = allData['GazePointX (MCSpx)'].diff()
allData['GazePointY (MCSpx)'] = allData['GazePointY (MCSpx)'].diff()
allData['GazePointX (MCSpx)'].fillna(method='bfill', inplace= True)
allData['GazePointY (MCSpx)'].fillna(method='bfill', inplace= True)
allData.head()

,ParticipantName,RecordingDuration,RecordingTimestamp,LocalTimeStamp,EyeTrackerTimestamp,MouseEventIndex,MouseEventX (MCSpx),MouseEventY (MCSpx),KeyPressEventIndex,StudioEventIndex,StudioEventIndex.1,StudioEvent,StudioEvent_B,FixationIndex,SaccadeIndex,GazeEventType,GazeEventType_B,GazeEventDuration,FixationPointX (MCSpx),FixationPointY (MCSpx),AbsoluteSaccadicDirection,RelativeSaccadicDirection,GazePointIndex,GazePointX (MCSpx),GazePointY (MCSpx),PupilLeft,PupilRight,ValidityLeft,ValidityRight,StudioEventTypeDiff,GazeEventTypeDiff,StudioEventIndex_B,StudioEventIndex_B.1,Unnamed: 11
0,Proband_30_D1,3520531,377,14:45:45.208,1.457961e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,NaN,4.0,Saccade,Saccade,233,NaN,NaN,NaN,NaN,22.0,81.0,239.0,"2,14","2,19",0.0,0.0,Agree_0_unstated,Agree_Saccade,NaN,NaN,NaN
1,Proband_30_D1,3520531,394,14:45:45.225,1.457961e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,NaN,4.0,Saccade,Saccade,233,NaN,NaN,NaN,NaN,23.0,81.0,239.0,"2,02","2,15",0.0,0.0,Agree_0_unstated,Agree_Saccade,NaN,NaN,NaN
2,Proband_30_D1,3520531,411,14:45:45.242,1.457961e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,NaN,4.0,Saccade,Saccade,233,NaN,NaN,NaN,NaN,24.0,72.0,120.0,"1,93","2,01",0.0,0.0,Agree_0_unstated,Agree_Saccade,NaN,NaN,NaN
3,Proband_30_D1,3520531,428,14:45:45.258,1.457961e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,NaN,4.0,Saccade,Saccade,233,NaN,NaN,NaN,NaN,25.0,-66.0,156.0,"1,93","1,94",4.0,0.0,Agree_0_unstated,Agree_Saccade,NaN,NaN,NaN
4,Proband_30_D1,3520531,444,14:45:45.275,1.457961e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,NaN,4.0,Saccade,Saccade,233,NaN,NaN,NaN,NaN,26.0,71.0,184.0,"2,58","2,06",0.0,0.0,Agree_0_unstated,Agree_Saccade,NaN,NaN,NaN


# Chi Square 

## Annotation A

In [9]:
import scipy.stats as stats
#Selecting p value as 0.10
#Ho: There is no relation  between variables
#Ha: There is a relationship
#Result p value should be greater than 0.10 to accept null hypothesis

In [10]:
pd.crosstab(allData['StudioEvent'],allData['GazeEventType'])

GazeEventType,Fixation,Saccade,Unclassified
StudioEvent,,,
0_Unstated,350,53,2
0_unstated,30048,7534,5054
1_Scanning,116211,50252,13231
2_Skimming,33232,13461,3682
3_Reading,80487,14911,2579
4_MediaView,3002,975,197
5_Unknown,157357,78184,119557


In [11]:
def drop_unstated(series):
    if series.lower() == '0_Unstated' or series.lower() == '0_unstated':
        return 
    else:
        return series

In [12]:
allData['Studio_Events'] = allData['StudioEvent'].apply(drop_unstated)

In [13]:
studio_crosstab = pd.crosstab(allData['Studio_Events'],allData['GazeEventType'])
studio_crosstab

GazeEventType,Fixation,Saccade,Unclassified
Studio_Events,,,
1_Scanning,116211,50252,13231
2_Skimming,33232,13461,3682
3_Reading,80487,14911,2579
4_MediaView,3002,975,197
5_Unknown,157357,78184,119557


In [14]:
stats.chi2_contingency(studio_crosstab)

(94640.53405167622,
 0.0,
 8,
 array([[102038.05453371,  41251.15070753,  36404.79475876],
        [ 28605.11200783,  11564.25210019,  10205.63589197],
        [ 55635.59422713,  22491.925122  ,  19849.48065088],
        [  2370.17841232,    958.19728568,    845.624302  ],
        [201640.06081901,  81517.4747846 ,  71940.46439639]]))

In [15]:
#Ho is rejected bc p value is less than 0.05
#There is a relation between Gaze event type and Studio events

### Using 10% of data

In [16]:
#Running the same test with 10% of data

subset_allData = allData.sample(frac=0.1)
len(subset_allData)

73036

In [17]:
subset_allData['Studio_Events'] = subset_allData['StudioEvent'].apply(drop_unstated)
subset_allData.head()

,ParticipantName,RecordingDuration,RecordingTimestamp,LocalTimeStamp,EyeTrackerTimestamp,MouseEventIndex,MouseEventX (MCSpx),MouseEventY (MCSpx),KeyPressEventIndex,StudioEventIndex,StudioEventIndex.1,StudioEvent,StudioEvent_B,FixationIndex,SaccadeIndex,GazeEventType,GazeEventType_B,GazeEventDuration,FixationPointX (MCSpx),FixationPointY (MCSpx),AbsoluteSaccadicDirection,RelativeSaccadicDirection,GazePointIndex,GazePointX (MCSpx),GazePointY (MCSpx),PupilLeft,PupilRight,ValidityLeft,ValidityRight,StudioEventTypeDiff,GazeEventTypeDiff,StudioEventIndex_B,StudioEventIndex_B.1,Unnamed: 11,Studio_Events
23375,Proband_27_D1,3272639,387526,09:33:42.394,1.457683e+15,98.0,150.0,18.0,237.0,52.0,NaN,5_Unknown,1_Scanning,890.0,1317.0,Fixation,Fixation,467,45.0,238.0,"191,77","68,08",23062.0,-4.0,7.0,"2,45","2,65",0.0,0.0,Disagree_5_Unknown_1_Scanning,Agree_Fixation,52.0,NaN,NaN,5_Unknown
2756,Proband_35_D3,2669577,46516,13:38:43.934,1.458216e+15,5.0,231.0,280.0,NaN,6.0,NaN,5_Unknown,3_Reading,124.0,141.0,Fixation,Fixation,200,66.0,372.0,"107,28","31,19",2767.0,14.0,-6.0,"2,82","2,81",0.0,0.0,Disagree_5_Unknown_3_Reading,Agree_Fixation,2.0,NaN,NaN,5_Unknown
9469,Proband_29_D3,2343337,157543,13:23:15.113,1.457956e+15,13.0,170.0,349.0,38.0,NaN,NaN,0_unstated,0_unstated,488.0,578.0,Fixation,Fixation,300,176.0,323.0,"64,27","71,53",9441.0,3.0,3.0,"3,34","3,19",0.0,0.0,Agree_0_unstated,Agree_Fixation,NaN,NaN,NaN,None
29103,Proband_37_D2,2578589,476814,16:29:45.961,1.458572e+15,51.0,352.0,20.0,656.0,42.0,NaN,1_Scanning,3_Reading,980.0,2484.0,Fixation,Saccade,150,1245.0,619.0,"355,87","20,36",28428.0,10.0,5.0,"2,56","2,71",0.0,0.0,Disagree_1_Scanning_3_Reading,Disagree_Fixation_Saccade,53.0,NaN,NaN,1_Scanning
6051,Proband_27_D1,3272639,102859,09:28:57.727,1.457683e+15,30.0,368.0,140.0,36.0,24.0,NaN,5_Unknown,3_Reading,239.0,277.0,Unclassified,Unclassified,1370,486.0,143.0,"8,84","353,38",6035.0,0.0,0.0,"2,36","2,82",4.0,4.0,Disagree_5_Unknown_3_Reading,Agree_Unclassified,15.0,NaN,NaN,5_Unknown


In [18]:
studio_crosstabsub = pd.crosstab(subset_allData['Studio_Events'],subset_allData['GazeEventType'])
studio_crosstabsub

GazeEventType,Fixation,Saccade,Unclassified
Studio_Events,,,
1_Scanning,11548,5017,1367
2_Skimming,3292,1365,363
3_Reading,8052,1472,247
4_MediaView,291,98,18
5_Unknown,15799,7874,11991


In [19]:
stats.chi2_contingency(studio_crosstabsub)

(9413.771223489397,
 0.0,
 8,
 array([[10161.13649446,  4125.24103846,  3645.62246708],
        [ 2844.57423613,  1154.8466436 ,  1020.57912027],
        [ 5536.72009187,  2247.81007065,  1986.46983749],
        [  230.62583946,    93.6299968 ,    82.74416374],
        [20208.94333808,  8204.47225049,  7250.58441143]]))

In [20]:
#Results are same, reject Ho hypothesis 

## Annotation B

In [21]:
pd.crosstab(allData['StudioEvent_B'],allData['GazeEventType'])

GazeEventType,Fixation,Saccade,Unclassified
StudioEvent_B,,,
0_unstated,31394,7975,5529
1_Scanning,75932,39644,11546
2_Skimming,37637,33514,8785
3_Reading,167573,44917,12455
4_MediaView,7311,2366,590
5_Unknown,100840,36954,105397


In [22]:
allData['Studio_Events_B'] = allData['StudioEvent_B'].apply(drop_unstated)

In [23]:
studio_crosstab_B = pd.crosstab(allData['Studio_Events_B'],allData['GazeEventType'])
studio_crosstab_B

GazeEventType,Fixation,Saccade,Unclassified
Studio_Events_B,,,
1_Scanning,75932,39644,11546
2_Skimming,37637,33514,8785
3_Reading,167573,44917,12455
4_MediaView,7311,2366,590
5_Unknown,100840,36954,105397


In [24]:
stats.chi2_contingency(studio_crosstab_B)

(148272.5882254324,
 0.0,
 8,
 array([[ 72196.23690626,  29189.65074599,  25736.11234775],
        [ 45397.95152168,  18354.83961888,  16183.20885944],
        [127752.72974684,  51651.68882694,  45540.58142622],
        [  5830.92434289,   2357.50022977,   2078.57542734],
        [138115.15748234,  55841.32057841,  49234.52193925]]))

### Using 10% of data

In [25]:
subset_allData_B = allData.sample(frac=0.1)
len(subset_allData_B)

73036

In [26]:
subset_allData_B['Studio_Events_B'].value_counts()

5_Unknown      24368
3_Reading      22457
1_Scanning     12752
2_Skimming      8002
4_MediaView     1039
Name: Studio_Events_B, dtype: int64

In [27]:
studio_crosstabsub_B = pd.crosstab(subset_allData_B['Studio_Events_B'],subset_allData_B['GazeEventType'])
studio_crosstabsub_B

GazeEventType,Fixation,Saccade,Unclassified
Studio_Events_B,,,
1_Scanning,7518,4013,1221
2_Skimming,3791,3316,895
3_Reading,16862,4340,1255
4_MediaView,715,251,73
5_Unknown,10222,3714,10432


In [28]:
stats.chi2_contingency(studio_crosstabsub_B)

(14447.180107407497,
 0.0,
 8,
 array([[ 7267.84831968,  2905.42959573,  2578.72208458],
        [ 4560.64321315,  1823.18441225,  1618.1723746 ],
        [12799.09580577,  5116.6273864 ,  4541.27680783],
        [  592.16549593,   236.72689382,   210.10761025],
        [13888.24716547,  5552.0317118 ,  4927.72112274]]))

# ANOVA 

In [29]:
import scipy.stats as stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols 

## GazaePointX

### SciPy

In [30]:
scanningData_gpx = allData.loc[allData['StudioEvent']=='1_Scanning',['GazePointX (MCSpx)','StudioEvent']]
skimmingData_gpx = allData.loc[allData['StudioEvent']=='2_Skimming',['GazePointX (MCSpx)','StudioEvent']]
readingData_gpx = allData.loc[allData['StudioEvent']=='3_Reading',['GazePointX (MCSpx)','StudioEvent']]
mediaData_gpx = allData.loc[allData['StudioEvent']=='4_MediaView',['GazePointX (MCSpx)','StudioEvent']]
unknownData_gpx = allData.loc[allData['StudioEvent']=='5_Unknown',['GazePointX (MCSpx)','StudioEvent']]

In [31]:
stats.f_oneway(scanningData_gpx['GazePointX (MCSpx)'][scanningData_gpx['StudioEvent'] == '1_Scanning'], 
             skimmingData_gpx['GazePointX (MCSpx)'][skimmingData_gpx['StudioEvent'] == '2_Skimming'],
             readingData_gpx['GazePointX (MCSpx)'][readingData_gpx['StudioEvent'] == '3_Reading'],
              mediaData_gpx['GazePointX (MCSpx)'][mediaData_gpx['StudioEvent'] == '4_MediaView'],
              unknownData_gpx['GazePointX (MCSpx)'][unknownData_gpx['StudioEvent'] == '5_Unknown'])

F_onewayResult(statistic=1.4002289112100026, pvalue=0.23100154560778396)

### StatsModels

In [32]:
df = allData.loc[:,['GazePointX (MCSpx)','StudioEvent']]
df = df.rename(columns={'GazePointX (MCSpx)': 'GazePointX'})
df['Studio_Events'] = df['StudioEvent'].apply(drop_unstated)
df = df.drop(['StudioEvent'], axis=1)
df.head()

,GazePointX,Studio_Events
0,81.0,None
1,81.0,None
2,72.0,None
3,-66.0,None
4,71.0,None


In [33]:
results = ols('GazePointX ~ C(Studio_Events)', data=df).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             GazePointX   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.400
Date:                Thu, 05 Sep 2019   Prob (F-statistic):              0.231
Time:                        04:11:54   Log-Likelihood:            -3.5785e+06
No. Observations:              687318   AIC:                         7.157e+06
Df Residuals:                  687313   BIC:                         7.157e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           0.0560      0.104      0.538      0.591      -0.148       0.260
C(Studio_Events)[T.2_Skimming]     -0.0236      0.223     -0.106      0.915      -0.460       0.413
C(Studio_Events)[T.3_Reading]       0.0028      0.175      0.016      0.987      -0.341       0.346
C(Studio_Events)[T.4_MediaView]    -1.5884      0.691     -2.298      0.022      -2.943      -0.234
C(Studio_Events)[T.5_Unknown]      -0.0782      0.128     -0.612      0.541      -0.329       0.172
==============================================================================
Omnibus:                   490149.026   Durbin-Watson:                   1.912
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        832912576.733
Skew:                          -1.938   Prob(JB):                         0.00
Kurtosis:                     173.496   Cond. No.                         15.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## GazePointY

### SciPy


In [34]:
scanningData_gpy = allData.loc[allData['StudioEvent']=='1_Scanning',['GazePointY (MCSpx)','StudioEvent']]
skimmingData_gpy = allData.loc[allData['StudioEvent']=='2_Skimming',['GazePointY (MCSpx)','StudioEvent']]
readingData_gpy = allData.loc[allData['StudioEvent']=='3_Reading',['GazePointY (MCSpx)','StudioEvent']]
mediaData_gpy = allData.loc[allData['StudioEvent']=='4_MediaView',['GazePointY (MCSpx)','StudioEvent']]
unknownData_gpy = allData.loc[allData['StudioEvent']=='5_Unknown',['GazePointY (MCSpx)','StudioEvent']]

In [35]:
stats.f_oneway(scanningData_gpy['GazePointY (MCSpx)'][scanningData_gpy['StudioEvent'] == '1_Scanning'], 
             skimmingData_gpy['GazePointY (MCSpx)'][skimmingData_gpy['StudioEvent'] == '2_Skimming'],
             readingData_gpy['GazePointY (MCSpx)'][readingData_gpy['StudioEvent'] == '3_Reading'],
              mediaData_gpy['GazePointY (MCSpx)'][mediaData_gpy['StudioEvent'] == '4_MediaView'],
              unknownData_gpy['GazePointY (MCSpx)'][unknownData_gpy['StudioEvent'] == '5_Unknown'])

F_onewayResult(statistic=2.0062817142509295, pvalue=0.09066336630340685)

### StatsModels

In [36]:
df_gpy = allData.loc[:,['GazePointY (MCSpx)','StudioEvent']]
df_gpy = df_gpy.rename(columns={'GazePointY (MCSpx)': 'GazePointY'})
df_gpy['Studio_Events'] = df_gpy['StudioEvent'].apply(drop_unstated)
df_gpy = df_gpy.drop(['StudioEvent'], axis=1)
df_gpy.head()

,GazePointY,Studio_Events
0,239.0,None
1,239.0,None
2,120.0,None
3,156.0,None
4,184.0,None


In [37]:
results = ols('GazePointY ~ C(Studio_Events)', data=df_gpy).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             GazePointY   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.006
Date:                Thu, 05 Sep 2019   Prob (F-statistic):             0.0907
Time:                        04:11:59   Log-Likelihood:            -3.6214e+06
No. Observations:              687318   AIC:                         7.243e+06
Df Residuals:                  687313   BIC:                         7.243e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                          -0.0271      0.111     -0.245      0.807      -0.244       0.190
C(Studio_Events)[T.2_Skimming]     -0.4981      0.237     -2.103      0.035      -0.962      -0.034
C(Studio_Events)[T.3_Reading]       0.0905      0.187      0.485      0.628      -0.275       0.456
C(Studio_Events)[T.4_MediaView]     0.4457      0.736      0.606      0.545      -0.996       1.888
C(Studio_Events)[T.5_Unknown]       0.1134      0.136      0.833      0.405      -0.153       0.380
==============================================================================
Omnibus:                   512345.126   Durbin-Watson:                   2.292
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        358522880.146
Skew:                          -2.324   Prob(JB):                         0.00
Kurtosis:                     114.792   Cond. No.                         15.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Pupil Left

In [38]:
allData['PupilLeft'] = allData['PupilLeft'].str.replace(',','.')
allData['PupilLeft'].head()

0    2.14
1    2.02
2    1.93
3    1.93
4    2.58
Name: PupilLeft, dtype: object

## SciPy

In [39]:
scanningData_pl = allData.loc[allData['StudioEvent']=='1_Scanning',['PupilLeft','StudioEvent']]
skimmingData_pl = allData.loc[allData['StudioEvent']=='2_Skimming',['PupilLeft','StudioEvent']]
readingData_pl = allData.loc[allData['StudioEvent']=='3_Reading',['PupilLeft','StudioEvent']]
mediaData_pl = allData.loc[allData['StudioEvent']=='4_MediaView',['PupilLeft','StudioEvent']]
unknownData_pl = allData.loc[allData['StudioEvent']=='5_Unknown',['PupilLeft','StudioEvent']]


In [40]:
stats.f_oneway(scanningData_pl['PupilLeft'][scanningData_pl['StudioEvent'] == '1_Scanning'], 
             skimmingData_pl['PupilLeft'][skimmingData_pl['StudioEvent'] == '2_Skimming'],
             readingData_pl['PupilLeft'][readingData_pl['StudioEvent'] == '3_Reading'],
              mediaData_pl['PupilLeft'][mediaData_pl['StudioEvent'] == '4_MediaView'],
              unknownData_pl['PupilLeft'][unknownData_pl['StudioEvent'] == '5_Unknown'])

F_onewayResult(statistic=2992.16026377699, pvalue=0.0)

## StatsModels

In [41]:
df_pl = allData.loc[:,['PupilLeft','StudioEvent']]
df_pl['Studio_Events'] = df_pl['StudioEvent'].apply(drop_unstated)
df_pl = df_pl.drop(['StudioEvent'], axis=1)
df_pl.head()

,PupilLeft,Studio_Events
0,2.14,None
1,2.02,None
2,1.93,None
3,1.93,None
4,2.58,None


In [42]:
#results = ols('PupilLeft ~ C(Studio_Events)', data=df_pl).fit()
#results.summary()

# Pupil Right

In [43]:
allData['PupilRight'] = allData['PupilRight'].str.replace(',','.')
allData['PupilRight'].head()

0    2.19
1    2.15
2    2.01
3    1.94
4    2.06
Name: PupilRight, dtype: object

## SciPy

In [44]:
scanningData_pr = allData.loc[allData['StudioEvent']=='1_Scanning',['PupilRight','StudioEvent']]
skimmingData_pr = allData.loc[allData['StudioEvent']=='2_Skimming',['PupilRight','StudioEvent']]
readingData_pr = allData.loc[allData['StudioEvent']=='3_Reading',['PupilRight','StudioEvent']]
mediaData_pr = allData.loc[allData['StudioEvent']=='4_MediaView',['PupilRight','StudioEvent']]
unknownData_pr = allData.loc[allData['StudioEvent']=='5_Unknown',['PupilRight','StudioEvent']]

In [45]:
stats.f_oneway(scanningData_pr['PupilRight'][scanningData_pr['StudioEvent'] == '1_Scanning'], 
             skimmingData_pr['PupilRight'][skimmingData_pr['StudioEvent'] == '2_Skimming'],
             readingData_pr['PupilRight'][readingData_pr['StudioEvent'] == '3_Reading'],
              mediaData_pr['PupilRight'][mediaData_pr['StudioEvent'] == '4_MediaView'],
              unknownData_pr['PupilRight'][unknownData_pr['StudioEvent'] == '5_Unknown'])

F_onewayResult(statistic=3278.516325877781, pvalue=0.0)

## StatsModels

In [46]:
df_pr = allData.loc[:,['PupilRight','StudioEvent']]
df_pr['Studio_Events'] = df_pr['StudioEvent'].apply(drop_unstated)
df_pr = df_pr.drop(['StudioEvent'], axis=1)
df_pr.head()

,PupilRight,Studio_Events
0,2.19,None
1,2.15,None
2,2.01,None
3,1.94,None
4,2.06,None


In [47]:
#results = ols('PupilRight ~ C(Studio_Events)', data=df_pr).fit()
#results.summary()